In [1]:
from datasets import load_dataset
import torch
import time
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GPTQConfig

In [2]:
prequantized_model = "merged_ft_model"
quantized_name = "gptq-ft_bloom1b1"
tokenizer = AutoTokenizer.from_pretrained(prequantized_model)
quantization_config = GPTQConfig(bits=4, dataset = "c4", tokenizer=tokenizer) #c4 local dir is not the actual c4 dataset but custom dataset (text-to-sql)

In [3]:
device_map = "cuda:0" if torch.cuda.is_available() else "cpu"
start = time.time()
quantized_model = AutoModelForCausalLM.from_pretrained(prequantized_model, device_map="auto", quantization_config=quantization_config)
end = time.time()
print("Total Seconds Taken to Quantize Using " + str(device_map) +": {}".format(end - start))

Repo card metadata block was not found. Setting CardData to empty.


Quantizing transformer.h blocks :   0%|          | 0/24 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/4 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/4 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/4 [00:00<?, ?it/s]

...

Quantizing layers inside the block:   0%|          | 0/4 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/4 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/4 [00:00<?, ?it/s]

Quantizing layers inside the block:   0%|          | 0/4 [00:00<?, ?it/s]

Total Seconds Taken to Quantize Using cuda:0: 284.3878264427185


In [4]:
quantized_model.to("cpu")
quantized_model.save_pretrained(quantized_name)

In [5]:
tokenizer.save_pretrained(quantized_name)

('gptq-ft_bloom1b1/tokenizer_config.json',
 'gptq-ft_bloom1b1/special_tokens_map.json',
 'gptq-ft_bloom1b1/tokenizer.json')

In [6]:
def print_param_precision(model):
  dtypes = {}
  for _, p in model.named_parameters():
      dtype = p.dtype
      if dtype not in dtypes:
          dtypes[dtype] = 0
      dtypes[dtype] += p.numel()
  total = 0
  for k, v in dtypes.items():
      total += v
  for k, v in dtypes.items():
      print(f"{k}, {v / 10**6:.4f} M, {v / total*100:.2f} %")

def print_trainable_parameters(model):
  # Count the total parameters
  total_params = sum(p.numel() for p in model.parameters())
  print(f"Total parameters: {total_params/10**6:.4f} M")

  # Count the trainable parameters
  trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print(f"Trainable parameters: {trainable_params/10**6:.4f} M")

In [8]:
print(f"{device_map} Memory Used: {quantized_model.get_memory_footprint() / 1024**2:.4f} MB")
print(f"--------------------------------------\n")
print(f"Parameters loaded for model {quantized_name}:")
print_trainable_parameters(quantized_model)
print(f"Data types for loaded model {quantized_name}:")
print_param_precision(quantized_model)

cuda:0 Memory Used: 1073.5664 MB
--------------------------------------

Parameters loaded for model gptq-ft_bloom1b1:
Total parameters: 385.5053 M
Trainable parameters: 385.5053 M
Data types for loaded model gptq-ft_bloom1b1:
torch.float16, 385.5053 M, 100.00 %
